In [ ]:
from PIL import Image, ImageDraw

PADDING = 0
OUTPUT_SCALE = 1
ERROR_THRESHOLD = 7


def weighted_average(hist):
    total = sum(hist)
    value, error = 0, 0
    if total > 0:
        value = sum(i * x for i, x in enumerate(hist)) / total
        error = sum(x * (value - i) ** 2 for i, x in enumerate(hist)) / total
        error = error ** 0.5
    return value, error


def color_from_histogram(hist):
    r, re = weighted_average(hist[:256])
    g, ge = weighted_average(hist[256:512])
    b, be = weighted_average(hist[512:768])
    e = re * 0.2989 + ge * 0.5870 + be * 0.1140
    return (int(r), int(g), int(b)), e


class QuadtreeNode(object):

    def __init__(self, img, box, depth):
        self.box = box
        self.depth = depth
        self.children = None
        self.leaf = False

        image = img.crop(box)
        self.width, self.height = image.size
        hist = image.histogram()
        self.color, self.error = color_from_histogram(hist)

    def is_leaf(self):
        return self.leaf

    def split(self, img):
        l, t, r, b = self.box
        lr = l + (r - l) / 2
        tb = t + (b - t) / 2
        tl = QuadtreeNode(img, (l, t, lr, tb), self.depth+1)
        tr = QuadtreeNode(img, (lr, t, r, tb), self.depth+1)
        bl = QuadtreeNode(img, (l, tb, lr, b), self.depth+1)
        br = QuadtreeNode(img, (lr, tb, r, b), self.depth+1)
        self.children = [tl, tr, bl, br]

    def to_image(self, x=0, y=0, img=None):
        """Converts this node and its children to an image"""
        if img is None:
            img = Image.new('RGB', (self.width, self.height))

        if self.children is None:
            # If this is a leaf node, fill the corresponding area of the image with the node's color
            for i in range(x, x + self.width):
                for j in range(y, y + self.height):
                    img.putpixel((i, j), self.color)
        else:
            # If this is an internal node, process each of the children
            half_width = self.width // 2
            half_height = self.height // 2
            self.children[0].to_image(x, y, img)
            self.children[1].to_image(x + half_width, y, img)
            self.children[2].to_image(x, y + half_height, img)
            self.children[3].to_image(x + half_width, y + half_height, img)

        return img


class Quadtree(object):

    def __init__(self, image, max_depth=1024):
        self.root = QuadtreeNode(image, image.getbbox(), 0)
        self.width, self.height = image.size
        self.max_depth = 0

        self._build_tree(image, self.root, max_depth)

    def _build_tree(self, image, node, max_depth):
        if (node.depth >= max_depth) or (node.error <= ERROR_THRESHOLD):
            if node.depth > self.max_depth:
                self.max_depth = node.depth
            node.leaf = True
            return

        node.split(image)
        for child in node.children:
            self._build_tree(image, child, max_depth)

    def get_leaf_nodes(self, depth):
        def get_leaf_nodes_recusion(tree, node, depth, func):
            if node.leaf is True or node.depth == depth:
                func(node)
            elif node.children is not None:
                for child in node.children:
                    get_leaf_nodes_recusion(tree, child, depth, func)

        if depth > self.max_depth:
            raise ValueError('A depth larger than the trees depth was given')

        leaf_nodes = []
        get_leaf_nodes_recusion(self, self.root, depth, leaf_nodes.append)
        return leaf_nodes

    def _create_image_from_depth(self, depth):
        m = OUTPUT_SCALE
        dx, dy = (PADDING, PADDING)
        image = Image.new('RGB', (int(self.width * m + dx),
                                  int(self.height * m + dy)))
        draw = ImageDraw.Draw(image)
        draw.rectangle((0, 0, self.width * m + dx,
                        self.height * m + dy), (0, 0, 0))

        leaf_nodes = self.get_leaf_nodes(depth)
        for node in leaf_nodes:
            l, t, r, b = node.box
            box = (l * m + dx, t * m + dy, r * m - 1, b * m - 1)
            draw.rectangle(box, node.color)
        return image

    def render_at_depth(self, depth=0):
        if depth > self.max_depth:
            raise ValueError('A depth larger than the trees depth was given')

        image = self._create_image_from_depth(depth)
        image.show()

    def create_gif(self, file_name, duration=1000, loop=0):
        images = []
        end_product_image = self._create_image_from_depth(self.max_depth)
        for i in range(self.max_depth):
            image = self._create_image_from_depth(i)
            images.append(image)
        for _ in range(4):
            images.append(end_product_image)
        images[0].save(
            file_name,
            save_all=True,
            append_images=images[1:],
            duration=duration, loop=loop)
    
    def calculate_size(self):
        def calculate_size_recursion(node):
            size = 1 
            if node.children is not None:
                for child in node.children:
                    size += calculate_size_recursion(child)
            return size

        return calculate_size_recursion(self.root)

    def compression_ratio(self):
        input_size = self.width * self.height * 3  # 3 bytes per pixel for RGB images
        output_size = self.calculate_size() * 24  # Assume each node takes up 24 bytes
        compression_ratio = input_size / output_size

        return compression_ratio
    
    def to_image(self):
        """Converts the quadtree to an image"""
        return self.root.to_image()


We will perform 4:2:0 chroma subsampling on the image, which reduces the resolution of the color information in the image while keeping the resolution of the brightness information the same. This can significantly reduce the size of the image without a noticeable loss in quality, because the human eye is less sensitive to changes in color than it is to changes in brightness.

In [ ]:
# Open the image and convert it to the YCbCr color space
image = Image.open('./Images/flowers.jpg').convert('YCbCr')

# Perform 4:2:0 chroma subsampling
y, cb, cr = image.split()
cb = cb.resize((cb.width // 2, cb.height // 2), Image.LANCZOS)
cr = cr.resize((cr.width // 2, cr.height // 2), Image.LANCZOS)
cb = cb.resize(image.size, Image.LANCZOS)
cr = cr.resize(image.size, Image.LANCZOS)

# Recombine the channels
image = Image.merge('YCbCr', (y, cb, cr)).convert('RGB')
tree = Quadtree(image, max_depth=10)

print(f'Compression ratio: {tree.compression_ratio()}')

Comparing different Compression Techniques and the time it takes along with the compression ratio

In [55]:
import time
import os

image = Image.open('../Images/flowers.jpg').convert('RGB')

# Compress using quadtree
start_time = time.time()
tree = Quadtree(image, max_depth=10)
tree_time = time.time() - start_time


# Compress using JPEG
start_time = time.time()
image.save('image.jpg', 'JPEG')
jpeg_time = time.time() - start_time
jpeg_size = os.path.getsize('image.jpg')

# Compress using WebP
start_time = time.time()
image.save('image.webp', 'WebP')
webp_time = time.time() - start_time
webp_size = os.path.getsize('image.webp')

# Print the results
print(f"Quadtree compression time: {tree_time} seconds, Compression ratio: {tree.compression_ratio()}")
print(f"JPEG compression time: {jpeg_time} seconds, Compression ratio: {(image.width * image.height * 3) / jpeg_size}")
print(f"WebP compression time: {webp_time} seconds, Compression ratio: {(image.width * image.height * 3) / webp_size}")

Quadtree compression time: 21.112312078475952 seconds, Compression ratio: 10.435104810084848
JPEG compression time: 0.04259681701660156 seconds, Compression ratio: 21.93008324239809
WebP compression time: 1.0626163482666016 seconds, Compression ratio: 38.18624752568574


In [56]:
tree.render_at_depth(10)
tree.max_depth
tree.create_gif('flowers.gif', duration=1000, loop=0)

Opening in existing browser session.
